# Coursework: Optimisation of a fantasy football team

The coursework is described in detail in the documentation provided on Moodle. This notebook contains some code for basic functions that read in the data file and define the solution/constraint checker that you must use to check your final solution.

As noted in the coursework, you don't have to use Python or DEAP to tackle this. However, the practicals have covered a lot of functionality that will be useful so you should find that the DEAP libraries provide a quick way to start and will save you some time in writing code.

## Important Information

If you use another language, then you should write out your solution to a csv file as a comma separated list of 0,1s (one value per row) indicating which players are included, and use the code provided in this notebook to read it in and check it. You report should include the screenshot of the  output from the function provided in this notebook, and *not* your own version of the function



# Data
The code below reads in the datafile and calculates the number of players available.  
Change the filepath to your local drive.

The file is sorted by player type. As I may check your solution **DO NOT** sort the file or alter it in any way as my code will expect to see it in this format.

Feel free to browse the file and analyse the data in any way you think might be useful

In [1084]:
#import some standard python packages that will be useful
import array
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import pandas as pd


In [1085]:
# THIS FUNCTION READS THE DATA FILE CONTAINING THE INFORMATION RE EACH PLAYER

# read data
data = (pd.read_csv("clean-data.csv")
        .reset_index(drop=True))

global num_players
num_players = len(data.index)

print("num possible players is %s" % (num_players))


num possible players is 523


# Helpful data
The code below extracts some useful information from the data that will be useful to you when writing your program. In particular:

- a list containing the **points** per player:  e.g. points[i] refers to the **points** associated with player *i*
- a list containing the **cost** per player: e.g. cost[i] refers to the **cost** associated with player *i*
- a list **gk** which indicates which player is a *goal-keeper*. The list is the same length as the number of players. gk[i]=0 if player *i* is not a goal-keeper; gk[i]=1 if player *i* is a goal-keeper
- a list **mid** which indicates which player is a *midfielder*. The list is the same length as the number of players. mid[i]=0 if player *i* is not a mid-fielder; mid[i]=1 if player *i* is a midfielder
- a list **defe** which indicates which player is a *defender*. The list is the same length as the number of players. defe[i]=0 if player *i* is not a defender; defe[i]=1 if player *i* is a defender
- a list **stri** which indicates which player is a *striker*. The list is the same length as the number of players. stri[i]=0 if player *i* is not a striker; stri[i]=1 if player *i* is a striker

In [1086]:
# HELPFUL DATA 
# these can be used for calculating points and costs and are also used in the constraint_checking function
points = data['Points'] 
cost = data['Cost']
    

# create lists with all elements initialised to 0
gk = np.zeros(num_players)
mid = np.zeros(num_players)
defe = np.zeros(num_players)
stri = np.zeros(num_players)

for i in range(num_players):
    if data['Position'][i] == 'GK':
        gk[i] = 1
    elif data['Position'][i] == 'DEF':
        defe[i] = 1
    elif data['Position'][i] == 'MID':
        mid[i] = 1
    elif data['Position'][i] == 'STR':
        stri[i]=1
  

# Solution and constraint checker function

You are free to represent an individiual in any way you wish. However, at the end of the evolutionary run, you *must* convert your solution to a list of length *num_players* in which each element is either 0 or 1. An element *i* should be set to 0 if player *i* is not included in the team, and to 1 if player *is* **is** included in the team.

You *must* call this function with your best solution and include a screen shot of the output in your report.

In [1087]:
# check the constraints
# the function MUST be passed a list of length num_players in which each bit is set to 0 or 1


def check_constraints(individual):
     
    broken_constraints = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints+=1
        print("total players is %s " %(c1))
        
    
    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual)) 
    if c2 > 100:
        broken_constraints+=1
        print("cost is %s " %(c2))
    
    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints+=1
        print("goalies is %s " %(c3))
    
    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints+=1
        print("DEFE is %s " %(c4))
            
    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3: 
        broken_constraints+=1
        print("MID is %s " %(c5))
        
    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1: 
        broken_constraints+=1
        print("STR is %s " %(c6))
        
    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]
    
    totalpoints = np.sum(np.multiply(points, individual))
        
        
    print("total broken constraints: %s" %(broken_constraints))
    print("total points: %s" %(totalpoints))
    print("total cost is %s" %(c2))
    print("selected players are %s" %(selectedPlayers))
    
    return broken_constraints, totalpoints

In [1088]:
global team_size
team_size = 11
# permutation  approach

global list_of_goal_keepers
list_of_goal_keepers = []
global list_of_defenders
list_of_defenders = []
global list_of_midfielders
list_of_midfielders = []
global list_of_strikers
list_of_strikers = []
# Create individuals as integer permutation representation
def get_positions_of_players(num_players):
    for i in range(num_players):
        # Lists start from index 0 so direct representation would be i+1
        if data['Position'][i] == 'GK':
            list_of_goal_keepers.append(i)
        elif data['Position'][i] == 'DEF':
            list_of_defenders.append(i)
        elif data['Position'][i] == 'MID':
            list_of_midfielders.append(i)
        elif data['Position'][i] == 'STR':
            list_of_strikers.append(i)

In [1089]:
# check the constraints
# the function MUST be passed a list of length num_players in which each bit is set to 0 or 1


def check_constraints_modified(individual):

    # Do not need to check if there are duplicates as the initialization takes care of it
    broken_constraints = [0,0,0,0,0,0]
    broken_constraints_num = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints[0] = 1
        broken_constraints_num += 1
        print("total players is %s " %(c1))


    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual))
    if c2 > 100:
        broken_constraints[1] = 1
        broken_constraints_num += 1
        print("cost is %s " %(c2))

    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints[2] = 1
        broken_constraints_num += 1
        print("goalies is %s " %(c3))

    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints[3] = 1
        broken_constraints_num += 1
        print("DEFE is %s " %(c4))

    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3:
        broken_constraints[4] = 1
        broken_constraints_num += 1
        print("MID is %s " %(c5))

    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1:
        broken_constraints[5] = 1
        broken_constraints_num += 1
        print("STR is %s " %(c6))

    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]

    totalpoints = np.sum(np.multiply(points, individual))


    print("total broken constraints: %s" %(broken_constraints_num))
    print("total points: %s" %(totalpoints))
    print("total cost is %s" %(c2))
    print("selected players are %s" %(selectedPlayers))

    return broken_constraints, totalpoints

def check_constraints_modified_no_print(individual):

    # Do not need to check if there are duplicates as the initialization takes care of it
    broken_constraints = [0,0,0,0,0,0]
    broken_constraints_num = 0

    # exactly 11 players - 0
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints[0] = 1
        broken_constraints_num += 1


    #need cost <= 100" - 1
    c2 = np.sum(np.multiply(cost, individual))
    if c2 > 100:
        broken_constraints[1] = 1
        broken_constraints_num += 1

    # need only 1 GK - 2 
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints[2] = 1
        broken_constraints_num += 1

    # need less than 3-5 DEF" - 3 
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints[3] = 1
        broken_constraints_num += 1

    #need 3- 5 MID - 4
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3:
        broken_constraints[4] = 1
        broken_constraints_num += 1

    # need 1 -1 3 STR" - 5
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1:
        broken_constraints[5] = 1
        broken_constraints_num += 1

    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]

    totalpoints = np.sum(np.multiply(points, individual))

    return broken_constraints, totalpoints
def check_constraints_modified_no_print_no_cost(individual):

    # Do not need to check if there are duplicates as the initialization takes care of it
    broken_constraints = [0,0,0,0,0,0]
    broken_constraints_num = 0

    # exactly 11 players - 0
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints[0] = 1
        broken_constraints_num += 1

    # need only 1 GK - 2 
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints[2] = 1
        broken_constraints_num += 1

    # need less than 3-5 DEF" - 3 
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints[3] = 1
        broken_constraints_num += 1

    #need 3- 5 MID - 4
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3:
        broken_constraints[4] = 1
        broken_constraints_num += 1

    # need 1 -1 3 STR" - 5
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1:
        broken_constraints[5] = 1
        broken_constraints_num += 1

    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]

    totalpoints = np.sum(np.multiply(points, individual))

    return broken_constraints, totalpoints

In [1090]:
global another_representation
another_representation = {}

# choose a random element from a list
# randomly shuffle a sequence
from numpy.random import seed
from numpy.random import shuffle

# Permutation
def initialization_create_feasible_individual(icls, size, pInit):
    break_loop = False
    try:
        list_of_used_players = []
        # first create an individual with all bits set to 0
        ind = icls(np.zeros(size))
        broken_constraint_array = [1]*5
        #  There must be exactly 11 players in the team
        item_indices = [-1]*11  # individual has to contain exactly 11 players
        # The total cost of the team must be less than or equal to £100
        #  You can’t pick the same player more than once (i.e. all players in a team are unique)
        pos = 0
        while 1 in broken_constraint_array:
            
            broken_constraint_array, totalpoints = check_constraints_runtime(team_size, size, item_indices, icls)
            if 1 not in broken_constraint_array:
                break
            if broken_constraint_array[2]:
                # Need exactly 1 GK
                shuffle(list_of_goal_keepers)
                while list_of_goal_keepers[0] in list_of_used_players:
                    shuffle(list_of_goal_keepers)
                item_indices[pos]=list_of_goal_keepers[0]
                list_of_used_players.append(item_indices[pos])
                pos += 1
            if broken_constraint_array[3]:
                # Need at least 3 DEF (up to 5)
                for iteration1 in range(3):
                    while list_of_defenders[iteration1] in list_of_used_players:
                        shuffle(list_of_defenders)
                    item_indices[pos] = list_of_defenders[iteration1]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
            if broken_constraint_array[4]:
                # Need at least 3 MIN (up to 5)
                shuffle(list_of_midfielders)
                for iteration2 in range(3):
                    while list_of_midfielders[iteration2] in list_of_used_players:
                         shuffle(list_of_midfielders)
                    item_indices[pos] = list_of_midfielders[iteration2]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
            if broken_constraint_array[5]:
                # Need at least 1 STR (up to 3)
                shuffle(list_of_strikers)
                while list_of_strikers[0] in list_of_used_players:
                    shuffle(list_of_strikers)
                item_indices[pos] = list_of_strikers[0]
                list_of_used_players.append(item_indices[pos])
                pos += 1
            if broken_constraint_array[0]:
                list_of_choices = []
                list_showing_players = []
                for j in range(11):
                    if item_indices[j] in list_of_goal_keepers:
                        list_showing_players.append("GK")
                    if item_indices[j] in list_of_defenders:
                        list_showing_players.append("DEF")
                    if item_indices[j] in list_of_midfielders:
                        list_showing_players.append("MIN")
                    if item_indices[j] in list_of_strikers:
                        list_showing_players.append("STR")
                def_players = list_showing_players.count("DEF")
                min_players = list_showing_players.count("MIN")
                str_players = list_showing_players.count("STR")
                if def_players < 5:
                    list_of_choices.append(1)
                if min_players < 5:
                    list_of_choices.append(2)
                if str_players < 3:
                    list_of_choices.append(3)
                shuffle(list_of_choices)
                choice = list_of_choices[0]
                if choice == 1:
                    random.shuffle(list_of_defenders)
                    while list_of_defenders[0] in list_of_used_players:
                        random.shuffle(list_of_defenders)
                    item_indices[pos] = list_of_defenders[0]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
                elif choice == 2:
                    shuffle(list_of_midfielders)
                    while list_of_midfielders[0] in list_of_used_players:
                        shuffle(list_of_midfielders)
                    item_indices[pos] = list_of_midfielders[0]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
                elif choice == 3:
                    shuffle(list_of_strikers)
                    while list_of_strikers[0] in list_of_used_players:
                        shuffle(list_of_strikers)
                    item_indices[pos] = list_of_strikers[0]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
        item_indices_ind = [0]*size
        for i in range(team_size):
                item = item_indices[i]
                if item != -1:
                    ind[item]=1
                    item_indices_ind[item] = 1
        #s = ','.join(str(x) for x in item_indices_ind)
        #another_representation.update({s : item_indices})
        return ind
    except IndexError as e:
        list_showing_players = []
        for j in range(11):
            if item_indices[j] in list_of_goal_keepers:
                list_showing_players.append("GK")
            if item_indices[j] in list_of_defenders:
                list_showing_players.append("DEF")
            if item_indices[j] in list_of_midfielders:
                list_showing_players.append("MIN")
            if item_indices[j] in list_of_strikers:
                list_showing_players.append("STR")
        print(f'caught {type(e)}: e')

    
def check_constraints_runtime(team_size, size, item_indices, icls):
     # first create an individual with all bits set to 0
    ind = icls(np.zeros(size))
    for i in range(team_size):
            item = item_indices[i]
            if item != -1:
                ind[item]=1
    broken_constraint_array, totalpoints = check_constraints_modified_no_print_no_cost(ind)
    return broken_constraint_array, totalpoints

In [1091]:
# https://github.com/fergaljd/pep_ga/blob/d7f0d5877009ea0bb2ff452d2454ba1e0ebfb4a1/selection.py
def _roulette_wheel(population, pointers, number_to_keep = 2):
    keep = []
    i = 0
    cum_fitness = population[i].fitness.values[0]
    rotation = 0
    while len(keep) < 10:
        rotation += 1
        for point in pointers:
            while cum_fitness < point:
                i += 1
                cum_fitness += population[i].fitness.values[0]
            #Picks should be unique!
            #if population[i] not in keep:
            keep.append(population[i])

    return keep


In [1092]:
# Selection method
# https://github.com/fergaljd/pep_ga/blob/d7f0d5877009ea0bb2ff452d2454ba1e0ebfb4a1/selection.py            
def stochastic_universal_sampling(individuals, number_to_keep=4):
    '''
    Similar to the proportional selection,
    every individual obtains a segment on a
    roulette wheel according to its fitness
    value.
    However, it is turned only one time with
    nballs where n is the number of individuals in the
    population.
    https://en.wikipedia.org/wiki/Stochastic_universal_sampling
    '''
    sum_fitnesses = sum([c.fitness.values[0] for c in individuals])
    pointer_spread = float(sum_fitnesses) / 100
    start = random.uniform(0, pointer_spread)
    pointers = [(start+i*pointer_spread) for i in range(100)]
    keep = _roulette_wheel(individuals, pointers, number_to_keep)
    return keep

#Selection Functions
def proportional_selection(population, individuals, variable_2):
    '''
    Proportional (roulette wheel) selection: To each
    individual an area on a roulette wheel is assigned
    depending on its fitness value. Individuals with
    higher value have a higher probability to be selected
    than individuals with lower fitness value.
    '''
    keep = stochastic_universal_sampling(individuals, 1)
    return keep

In [1093]:
# Crossover - to do
def cxTwoPoint_(ind1, ind2):
    """Executes a two-point crossover on the input :term:`sequence`
    individuals. The two individuals are modified in place and both keep
    their original length.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the Python
    base :mod:`random` module.
    """
     # 0 - 180 DEF -> from 3 to 5 players
    def_to_crossover_1 = []
    def_to_crossover_2 = []
    # 181 - 376 MID
    mid_to_crossover_1 = []
    mid_to_crossover_2 = []
    # 377 - 466 STR
    str_to_crossover_1 = []
    str_to_crossover_2 = []
    # 467 - 523 GK
    GK_to_crossover_1 = []
    GK_to_crossover_2 = []
    # get indices of players selected
    selectedPlayers_individual_1 = [idx for idx, element in enumerate(ind1) if element==1]
    selectedPlayers_individual_1_copy = selectedPlayers_individual_1
    selectedPlayers_individual_2 = [idx for idx, element in enumerate(ind2) if element==1]
    selectedPlayers_individual_2_copy = selectedPlayers_individual_2


    def_to_crossover_1,mid_to_crossover_1, str_to_crossover_1, GK_to_crossover_1 = select_genomes_to_crossover(selectedPlayers_individual_1, def_to_crossover_1,mid_to_crossover_1, str_to_crossover_1, GK_to_crossover_1)
    def_to_crossover_2,mid_to_crossover_2, str_to_crossover_2, GK_to_crossover_2 = select_genomes_to_crossover(selectedPlayers_individual_2, def_to_crossover_2,mid_to_crossover_2, str_to_crossover_2, GK_to_crossover_2)

    size_def = min(len(def_to_crossover_1), len(def_to_crossover_2)) 
    size_mid = min(len(mid_to_crossover_1), len(mid_to_crossover_2)) 
    size_str = min(len(str_to_crossover_1), len(str_to_crossover_2)) 
    size_gk = min(len(GK_to_crossover_1), len(GK_to_crossover_2))

    cxpoint_def = random.randint(1, size_def)
    cxpoint_mid = random.randint(1, size_mid)
    cxpoint_str = random.randint(1, size_str) 
    cxpoint_gk = random.randint(1, size_gk)

    def_to_crossoever = merge_two_list_no_duplicate(def_to_crossover_1, def_to_crossover_2)
    mid_to_crossoever = merge_two_list_no_duplicate(mid_to_crossover_1, mid_to_crossover_2)
    str_to_crossoever = merge_two_list_no_duplicate(str_to_crossover_1, str_to_crossover_2)
    gk_to_crossoever = merge_two_list_no_duplicate(GK_to_crossover_1, GK_to_crossover_2)

    selectedPlayers_individual_1 = crossoever(def_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "DEF")
    selectedPlayers_individual_1 = crossoever(mid_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "MID")
    selectedPlayers_individual_1 = crossoever(str_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "STR")
    selectedPlayers_individual_1 = crossoever(gk_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "GK")

    selectedPlayers_individual_2 = crossoever(def_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "DEF")
    selectedPlayers_individual_2 = crossoever(mid_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "MID")
    selectedPlayers_individual_2 = crossoever(str_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "STR")
    selectedPlayers_individual_2 = crossoever(gk_to_crossoever, selectedPlayers_individual_1, cxpoint_def, "GK")


    if selectedPlayers_individual_1 != selectedPlayers_individual_1_copy:
            print("horray")
    if selectedPlayers_individual_2 != selectedPlayers_individual_2_copy:
        print("horray")
    # get indices of players selected
    #selectedPlayers_individual_1 = [idx for idx, element in enumerate(ind1) if element==1]
    index_coutner = 0
    for index_1 in selectedPlayers_individual_1:
        for index, element in enumerate(ind1):
            if index_1 == index:
                ind1[index_1] = 1
                index_coutner = index_1
            elif index_1 != index and index_coutner <= index:
                ind1[index_1] = 0
    # get indices of players selected
    #selectedPlayers_individual_1 = [idx for idx, element in enumerate(ind1) if element==1]
    index_coutner = 0
    for index_1 in selectedPlayers_individual_2:
        for index, element in enumerate(ind2):
            if index_1 == index:
                ind1[index_1] = 1
                index_coutner = index_1
            elif index_1 != index and index_coutner <= index:
                ind1[index_1] = 0
    return ind1, ind2
    
    # Whether to produce to children and replace parents or one child and keep one parent 
    bollean = random.getrandbits(1)
    # Produce two children
    # if bollean == 0:

    #     # def_to_crossover_1_no_duplicetes = remove_duplicates(def_to_crossover_1, def_to_crossover_2)
    #     # mid_to_crossover_1_no_duplicetes = remove_duplicates(mid_to_crossover_1, mid_to_crossover_2)
    #     # str_to_crossover_1_no_duplicetes = remove_duplicates(str_to_crossover_1, str_to_crossover_2)
    #     # gk_to_crossover_1_no_duplicetes = remove_duplicates(GK_to_crossover_1, GK_to_crossover_2)
        
    #     # def_to_crossover_2_no_duplicetes = remove_duplicates(def_to_crossover_2, def_to_crossover_1)
    #     # mid_to_crossover_2_no_duplicetes = remove_duplicates(mid_to_crossover_2, mid_to_crossover_1)
    #     # str_to_crossover_2_no_duplicetes = remove_duplicates(str_to_crossover_2, str_to_crossover_1)
    #     # gk_to_crossover_2_no_duplicetes = remove_duplicates(GK_to_crossover_2, GK_to_crossover_1)

    #     # def_to_swap, number_of_list_def = random_selection(def_to_crossover_1_no_duplicetes, def_to_crossover_2_no_duplicetes, cxpoint_def)
    #     # mid_to_swap, number_of_list_mid = random_selection(mid_to_crossover_1_no_duplicetes, mid_to_crossover_2_no_duplicetes, cxpoint_mid)
    #     # str_to_swap, number_of_list_str = random_selection(str_to_crossover_1_no_duplicetes, str_to_crossover_2_no_duplicetes, cxpoint_str)
    #     # gk_to_swap, number_of_list_gk = random_selection(gk_to_crossover_1_no_duplicetes, gk_to_crossover_2_no_duplicetes, cxpoint_gk) 

    #     # remove_from_list_1, remove_from_list_2 = swap_genomes(number_of_list_def, def_to_swap, selectedPlayers_individual_2, selectedPlayers_individual_1, def_to_crossover_2, def_to_crossover_1)
    #     # remove_from_list_1, remove_from_list_2 = swap_genomes(number_of_list_mid, mid_to_swap, remove_from_list_2, remove_from_list_1, mid_to_crossover_2, mid_to_crossover_1) # issue
    #     # remove_from_list_1, remove_from_list_2 = swap_genomes(number_of_list_str, str_to_swap, remove_from_list_2, remove_from_list_1, str_to_crossover_2, str_to_crossover_1)
    #     # remove_from_list_1, remove_from_list_2 = swap_genomes(number_of_list_gk, gk_to_swap, remove_from_list_2, remove_from_list_1, GK_to_crossover_2, GK_to_crossover_1)
    
    #     
    # else:

    #     totalpoints_parent_1 = np.sum(np.multiply(points, selectedPlayers_individual_1_copy))
    #     totalpoints_parent_2 = np.sum(np.multiply(points, selectedPlayers_individual_2_copy))
    #     totalpoints_child_1 = np.sum(np.multiply(points, ind1))
    #     totalpoints_child_2 = np.sum(np.multiply(points, ind2))

    #     sorted_numbers = sorted([totalpoints_parent_1, totalpoints_parent_2, totalpoints_child_1, totalpoints_child_2], reverse=True)

    
        
        

def merge_two_list_no_duplicate(first_list, second_list):
    in_first = set(first_list)
    in_second = set(second_list)

    in_second_but_not_in_first = in_second - in_first

    result = first_list + list(in_second_but_not_in_first)
    return result

def crossoever(list_of_all_players_for_position, individual, number_of_players_replaced, position):
    already_used_players = []
    for i in range(number_of_players_replaced):
        while list_of_all_players_for_position[0] in individual:
            shuffle(list_of_all_players_for_position)
        
        if position == "DEF":
            # Random integer from 0 to 180
            new_genome = random.randint(0, 180)
            while new_genome not in individual:
                new_genome = random.randint(0, 180)
            index_of_genome = individual.index(new_genome)
            individual[index_of_genome] = list_of_all_players_for_position[0]
            already_used_players.append(list_of_all_players_for_position)
        elif position == "MID":
            # Random integer from 181 to 376
            new_genome = random.randint(181, 376)
            while new_genome not in individual:
                new_genome = random.randint(181, 376)
            index_of_genome = individual.index(new_genome)
            individual[index_of_genome] = list_of_all_players_for_position[0]
            already_used_players.append(list_of_all_players_for_position)
        elif position == "STR":
            # Random integer from 377 to 466
            new_genome = random.randint(377, 466)
            while new_genome not in individual:
                new_genome = random.randint(377, 466)
            index_of_genome = individual.index(new_genome)
            individual[index_of_genome] = list_of_all_players_for_position[0]
            already_used_players.append(list_of_all_players_for_position)
        elif position == "GK" :
            # 467 - 523 GK
            new_genome = random.randint(467, 523)
            while new_genome not in individual:
                new_genome = random.randint(467, 523)
            index_of_genome = individual.index(new_genome)
            individual[index_of_genome] = list_of_all_players_for_position[0]
            already_used_players.append(list_of_all_players_for_position)
    
    
def select_genomes_to_crossover(individual, def_to_crossover,mid_to_crossover, str_to_crossover, GK_to_crossover):
    # 
    for genome in individual:
        if genome >= 0 and  genome <= 180 :
            def_to_crossover.append(genome)
        elif genome >= 181 and  genome <= 376:
            mid_to_crossover.append(genome)
        elif genome >= 377 and  genome <= 466 :
            str_to_crossover.append(genome)
        elif genome >= 467 and  genome <= 523 :
            GK_to_crossover.append(genome)
    return def_to_crossover,mid_to_crossover, str_to_crossover, GK_to_crossover

def random_selection(list_to_choose_from_1, list_to_choose_from_2, number_of_things_to_choose):
    bollean = random.getrandbits(1)
    sample_list = []
    if bollean == 0:
        for item in range(number_of_things_to_choose):
            random.shuffle(list_to_choose_from_1)
            if list_to_choose_from_1[0] not in sample_list:
                sample_list.append(list_to_choose_from_1[0])
            else:
                while list_to_choose_from_1[0] in sample_list:
                    random.shuffle(list_to_choose_from_1)
                sample_list.append(list_to_choose_from_1[0])
        return sample_list, 1
    else:
        for item in range(number_of_things_to_choose):
            random.shuffle(list_to_choose_from_2)
            if list_to_choose_from_2[0] not in sample_list:
                sample_list.append(list_to_choose_from_2[0])
            else:
                while list_to_choose_from_2[0] in sample_list:
                    random.shuffle(list_to_choose_from_2)
                sample_list.append(list_to_choose_from_2[0])
        return sample_list, 2

def remove_items(items_to_swap, remove_from_list_1, list_to_choose_which_genome_to_swap):
    items_already_swapped = []
    for item in range(len(items_to_swap)):
        random.shuffle(list_to_choose_which_genome_to_swap)
        old_swap = list_to_choose_which_genome_to_swap[0]
        
        if old_swap not in items_already_swapped:
            index = remove_from_list_1.index(old_swap) # issue
            items_already_swapped.append(old_swap)
            remove_from_list_1[index] = items_to_swap[item]
        else:
            while old_swap in items_already_swapped:
                random.shuffle(list_to_choose_which_genome_to_swap)
                old_swap = list_to_choose_which_genome_to_swap[0]
            index = remove_from_list_1.index(old_swap) # issue
            remove_from_list_1[index] = items_to_swap[item]
            items_already_swapped.append(old_swap)
    return remove_from_list_1

def remove_duplicates(list_a, list_b):
    list_c = list_b
    for i in list_b[:]:
        if i in list_a:
            list_c.remove(i)
    return list_c

def swap_genomes(number_of_list, pos_to_swap, selectedPlayers_individual_2, selectedPlayers_individual_1, pos_to_crossover_2, pos_to_crossover_1):
    remove_from_list_1 = selectedPlayers_individual_1
    remove_from_list_2 = selectedPlayers_individual_2
    if  number_of_list == 1:
        remove_from_list_1 = remove_items(pos_to_swap, selectedPlayers_individual_1, pos_to_crossover_1)

    elif number_of_list == 2:
        remove_from_list_2 = remove_items(pos_to_swap, selectedPlayers_individual_2, pos_to_crossover_2)
    return remove_from_list_1, remove_from_list_2

In [1094]:
def mutFlipBit(individual, indpb):
    """Flip the value of the attributes of the input individual and return the
    mutant. The *individual* is expected to be a :term:`sequence` and the values of the
    attributes shall stay valid after the ``not`` operator is called on them.
    The *indpb* argument is the probability of each attribute to be
    flipped. This mutation is usually applied on boolean individuals.
    :param individual: Individual to be mutated.
    :param indpb: Independent probability for each attribute to be flipped.
    :returns: A tuple of one individual.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    """
    # get indices of players selected
    selectedPlayers_individual_1 = [idx for idx, element in enumerate(individual) if element==1]
    selectedPlayers_individual_1_copy = selectedPlayers_individual_1
   
    # 0 - 180 DEF -> from 3 to 5 players
    # 181 - 376 MID
    # 377 - 466 STR
    # 467 - 523 GK   
    
    selectedPlayers_individual_1 = depending_on_positon_mutate_gene(selectedPlayers_individual_1, indpb)
    if selectedPlayers_individual_1 != selectedPlayers_individual_1_copy:
        index_coutner = 0
        for index_1 in selectedPlayers_individual_1:
            for index, element in enumerate(individual):
                if index_1 == index:
                    individual[index_1] = 1
                    index_coutner = index_1
                elif index_1 != index and index_coutner <= index:
                    individual[index_1] = 0
    return individual,

def depending_on_positon_mutate_gene(selectedPlayers_individual_1, indpb):
    already_used_players = []
    for genome in selectedPlayers_individual_1:
        if random.random() < indpb:
            index_of_genome = selectedPlayers_individual_1.index(genome)
            if genome >= 0 and  genome <= 180 :
                # Random integer from 0 to 180
                new_genome = random.randint(0, 180)
                if new_genome not in already_used_players:
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
                else:
                    while new_genome in already_used_players:
                        new_genome = random.randint(0, 180)
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
            elif genome >= 181 and  genome <= 376:
                # Random integer from 181 to 376
                new_genome = random.randint(181, 376)
                if new_genome not in already_used_players:
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
                else:
                    while new_genome in already_used_players:
                        new_genome = random.randint(181, 376)
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
            elif genome >= 377 and  genome <= 466 :
                # Random integer from 377 to 466
                new_genome = random.randint(377, 466)
                if new_genome not in already_used_players:
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
                else:
                    while new_genome in already_used_players:
                        new_genome = random.randint(377, 466)
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
            elif genome >= 467 and  genome <= 523 :
                # Random integer from 467 to 523
                new_genome = random.randint(467, 523)
                if new_genome not in already_used_players:
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
                else:
                    while new_genome in already_used_players:
                        new_genome = random.randint(467, 523)
                    selectedPlayers_individual_1[index_of_genome] = new_genome
                    already_used_players.append(new_genome)
    return selectedPlayers_individual_1

In [1095]:
# DEFINE FITNESS FOR KNAPSACK
# fitness function definition - death penalty

def evalKnapsack1(individual):
    # In Eval Knapsack
    cost_for_individual = np.sum(np.multiply(cost, individual))
    points_for_individual = np.sum(np.multiply(points, individual))
    if  cost_for_individual > MAX_Cost:
        total_overbudget = cost_for_individual - MAX_Cost        # Bags that are overweight get a fitness of difference to max cost
        return points_for_individual - total_overbudget,
    return  points_for_individual,

In [1096]:

MAX_Cost = 100
global TOTAL_WEIGHT
TOTAL_WEIGHT = 0
for i in range(num_players):
    TOTAL_WEIGHT += points[i]

global TOTAL_PROFIT
TOTAL_PROFIT = 0
for i in range(num_players):
    TOTAL_PROFIT += cost[i]
# proportional penalty fitness function

def evalKnapsack2(individual):
    tweight = 0.0
    tvalue = 0.0
    penalty=0

    diff = min(100, abs(TOTAL_PROFIT-100))


    for item in range(num_players):
        if (individual[item]==1):
            tweight += cost[item]
            tvalue += points[item]
    
    if tweight > TOTAL_PROFIT:
        dist = abs(tweight - TOTAL_PROFIT) 
        penalty = tvalue * (dist/diff)
    else:
        penalty = 0
        
    return tvalue-penalty,


In [1097]:
def sort_population_by_fitness(population):
    return sorted(population, key = lambda c: c.fitness.values, reverse=True)
def eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
   
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))
        
        # Vary the pool of individuals
        offspring = varAnd(offspring, toolbox, cxpb, mutpb)
        
        for ind in offspring:
            if not ind.fitness.valid:
                invalid_ind.append(ind)
       
            
        # Evaluate the individuals with an invalid fitness
        #invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring
        # sort_population_by_fitness(population)
        # population[-len(offspring):] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook
def varAnd(population, toolbox, cxpb, mutpb):
    
    offspring = [toolbox.clone(ind) for ind in population]

    # Apply crossover and mutation on the offspring
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                        offspring[i])
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

    for i in range(len(offspring)):
        if random.random() < mutpb:
            offspring[i], = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values

    return offspring

In [1098]:
def selStochasticUniversalSampling(individuals, k, fit_attr="fitness"):
    """Select the *k* individuals among the input *individuals*.
    The selection is made by using a single random value to sample all of the
    individuals by choosing them at evenly spaced intervals. The list returned
    contains references to the input *individuals*.
    :param individuals: A list of individuals to select from.
    :param k: The number of individuals to select.
    :param fit_attr: The attribute of individuals to use as selection criterion
    :return: A list of selected individuals.
    This function uses the :func:`~random.uniform` function from the python base
    :mod:`random` module.
    """
    s_inds = sorted(individuals, key=attrgetter(fit_attr), reverse=True)
    sum_fits = sum(getattr(ind, fit_attr).values[0] for ind in individuals)

    distance = sum_fits / float(k)
    start = random.uniform(0, distance)
    points = [start + i*distance for i in range(k)]

    chosen = []
    for p in points:
        i = 0
        sum_ = getattr(s_inds[i], fit_attr).values[0]
        while sum_ < p:
            i += 1
            sum_ += getattr(s_inds[i], fit_attr).values[0]
        chosen.append(s_inds[i])

    return chosen
def selection_process(population, individuals, size):
   return selStochasticUniversalSampling(individuals, 10)
class attrgetter:
    """
    Return a callable object that fetches the given attribute(s) from its operand.
    After f = attrgetter('name'), the call f(r) returns r.name.
    After g = attrgetter('name', 'date'), the call g(r) returns (r.name, r.date).
    After h = attrgetter('name.first', 'name.last'), the call h(r) returns
    (r.name.first, r.name.last).
    """
    __slots__ = ('_attrs', '_call')

    def __init__(self, attr, *attrs):
        if not attrs:
            if not isinstance(attr, str):
                raise TypeError('attribute name must be a string')
            self._attrs = (attr,)
            names = attr.split('.')
            def func(obj):
                for name in names:
                    obj = getattr(obj, name)
                return obj
            self._call = func
        else:
            self._attrs = (attr,) + attrs
            getters = tuple(map(attrgetter, self._attrs))
            def func(obj):
                return tuple(getter(obj) for getter in getters)
            self._call = func

    def __call__(self, obj):
        return self._call(obj)

    def __repr__(self):
        return '%s.%s(%s)' % (self.__class__.__module__,
                              self.__class__.__qualname__,
                              ', '.join(map(repr, self._attrs)))

    def __reduce__(self):
        return self.__class__, self._attrs

In [1100]:
# this returns a single individual: this function has the probability pInit of initialsing as feasible:
# if it is set to 0, initialisation is all random. If it is 1, initialistion is all feasible
# Binary Representation


get_positions_of_players(num_players)
# create a toolbox
toolbox = base.Toolbox()
# define the fitness class and creare an individual class
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximization problem for the value not the cost
creator.create("Individual", list, fitness=creator.FitnessMax)
# USE THIS LINE IF YOU WANT TO USE THE CUSTOM INIT FUNCTION
toolbox.register("individual", initialization_create_feasible_individual, creator.Individual, num_players, 1.0)

#  a population consist of a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# register all operators we need with the toolbox
toolbox.register("evaluate", evalKnapsack2)
toolbox.register("mate", cxTwoPoint_)
toolbox.register("mutate", mutFlipBit, indpb=0.05)
toolbox.register("select",  selection_process, toolbox.population)
pop = toolbox.population(n=10)
for individual in pop:
         broken_constraint, total_poins = check_constraints_modified_no_print(individual)
         if broken_constraint[0] == 1:
             print("Problem")
#     else:
#         print("No PROBLEM")
# keep track of the single best solution found
hof = tools.HallOfFame(1)

# create a statistics object: we can log what ever statistics we want using this. We use the numpy Python library
# to calculate the stats and label them with convenient labels
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# run the algorithm: we need to tell it what parameters to use
# cxpb = crossover probability; mutpb = mutation probability; ngen = number of iterations
pop, log = eaSimple(pop, toolbox, cxpb=1.0, mutpb=50.0, ngen=200,
                               stats=stats, halloffame=hof, verbose=True)

best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list


# look in the logbook to see what generation this was found at
max = log.select("max")  # max fitness per generation stored in log
sorted_pop = sorted(pop, key=lambda ind: ind.fitness, reverse=True)
print(sorted_pop[0])
check_constraints(sorted_pop[0])
for i in range(100):  # set to ngen
    fit = max[i]
    if fit == best:
        break

print("max fitness found is %s at generation %s" % (best, i))

In [ ]:
def sim_anneal(tour):


    # and note the distance for this 
    best_tour = tour
    best_dist = total_distance(tour)


    # set the maximum number of tries
    max_tries = 10000
    
    # for info set some counters
    num_accept_inferiorSols = 0
    num_reject_inferiorSols = 0
    


    # create an list and add the fitness of the starting tour (this is just for the purpose of plotting later)
    fitness_vals = list()
    fitness_vals.append(best_dist)
    initial_length = best_dist

    start_temp = 500
    end_temp = 0
    coolingRate = 0.999
    
    current_temp = start_temp
    
    
    # now run the HC loop : the loop should 
    for i in range(max_tries-1):
        new_tour = apply_neighbourhood_swap(best_tour)
        new_tour_dist = total_distance(new_tour)
        if (new_tour_dist <= best_dist):
            #  new one is best - accept
            best_dist = new_tour_dist
            best_tour= new_tour.copy()
        else:
            # tour is worse so calculate probability of accept
            diff = new_tour_dist-best_dist
            prob = np.exp((-1*diff)/current_temp)
            r = random.uniform(0, 1)
            
            if (prob > r):
                # accept: replace current with new solution
                best_dist = new_tour_dist
                best_tour= new_tour.copy()
                num_accept_inferiorSols += 1
                
                # if you want you can uncomment this to print a running commentary on what's happening
                #print("gen %d accepted worse %s  temp %s prob %s" % (i, best_dist, current_temp, prob))
            else:
                num_reject_inferiorSols += 1
        # reduce temp
        current_temp = current_temp*coolingRate
        fitness_vals.append(best_dist)

    #print("accepted %s inferior, rejected %s inferior" % (num_accept_inferiorSols, num_reject_inferiorSols))
    
    print("Starting tour length %s Best found %s  improvement of %s" % (initial_length, best_dist, initial_length-best_dist))  
    
    # return the results
    return best_dist, best_tour, fitness_vals